# #12 Minimum Volatility Portfolio_scipy.optimize & cvxopt

[reference](http://henryquant.blogspot.com/2017/08/minimum-volatility-portfolio-python.html)

## 1. method using `scipy.optimize`

In [1]:
import pandas as pd
from pandas import Series, DataFrame
from pandas.tseries.offsets import Day, MonthEnd

import numpy as np
import sys

import matplotlib.pyplot as plt

from scipy.stats import rankdata
from scipy.stats import stats
from scipy.optimize import minimize

In [2]:
# setting the dataset
import yfinance as yf
tickers = ['MSFT', 'TLT', 'SPY', 'VTIP', 'VCLT', 'IAU', 'BCI']
df = yf.download(tickers, start='2017-06-01', end='2023-04-01')['Adj Close']
df.head()

[*********************100%***********************]  7 of 7 completed


,BCI,IAU,MSFT,SPY,TLT,VCLT,VTIP
Date,,,,,,,
2017-06-01,15.392171,24.420000,65.030838,218.480850,108.937904,73.038704,41.170406
2017-06-02,15.392171,24.580000,66.570793,219.208054,110.227760,73.720528,41.187050
2017-06-05,15.290110,24.620001,67.053185,219.046448,109.543358,73.363762,41.153759
2017-06-06,15.290110,24.900000,67.275848,218.346130,110.140038,73.736374,41.220337
2017-06-07,15.290110,24.780001,67.155251,218.750183,109.613579,73.585724,41.137112


In [3]:
ret = df.pct_change().dropna() * 100
ret

,BCI,IAU,MSFT,SPY,TLT,VCLT,VTIP
Date,,,,,,,
2017-06-02,0.000000,0.655200,2.368038,0.332845,1.184029,0.933510,0.040426
2017-06-05,-0.663073,0.162738,0.724629,-0.073723,-0.620898,-0.483944,-0.080828
2017-06-06,0.000000,1.137282,0.332071,-0.319712,0.544697,0.507897,0.161778
2017-06-07,0.000000,-0.481923,-0.179259,0.185051,-0.477990,-0.204309,-0.201903
2017-06-08,0.041708,-0.726393,-0.607839,0.049247,-0.264177,-0.172379,-0.060674
...,...,...,...,...,...,...,...
2023-03-27,0.988635,-0.907635,-1.493395,0.186982,-2.349088,-1.673160,-0.272422
2023-03-28,0.783161,0.835133,-0.416083,-0.224465,0.182101,-0.179115,0.021006
2023-03-29,-0.437106,-0.507611,1.918388,1.453483,-0.200893,0.692130,0.105048


In [4]:
cov = df.cov()
cov

,BCI,IAU,MSFT,SPY,TLT,VCLT,VTIP
BCI,10.749686,6.505213,161.215299,165.315240,-23.520090,-4.962049,5.749515
IAU,6.505213,23.186728,341.220230,296.775978,48.636282,34.722918,11.067567
MSFT,161.215299,341.220230,6354.801527,5768.973100,557.264683,503.773462,201.301259
SPY,165.315240,296.775978,5768.973100,5459.413336,374.998563,430.534659,184.230414
TLT,-23.520090,48.636282,557.264683,374.998563,319.920528,168.482334,14.725918
VCLT,-4.962049,34.722918,503.773462,430.534659,168.482334,106.872835,14.602770
VTIP,5.749515,11.067567,201.301259,184.230414,14.725918,14.602770,6.572558


In [5]:
def MinVol(covmat, lb, ub):
    def MinVol_obj(x):
        variance = x.T @ covmat @ x
        sigma = variance ** 0.5  # std dev.
        return sigma

    def weight_sum_constraint(x):
        return (x.sum() - 1.0)

    sz = covmat.shape[1]
    x0 = np.repeat(1/sz, sz)  # (initial) uniform weight
    lbound = np.repeat(lb, sz)
    ubound = np.repeat(ub, sz)
    bnds = tuple(zip(lbound, ubound))

    constraints = ({'type': 'eq', 'fun': weight_sum_constraint})
    options = ({'ftol': 1e-20, 'maxiter': 800})

    result = minimize(fun=MinVol_obj,
                      x0=x0,
                      method='SLSQP',
                      constraints=constraints,
                      options=options,
                      bounds=bnds)
    return result.x

In [6]:
MinVol(cov, 0, 1)

array([1.41338339e-01, 3.68291545e-14, 0.00000000e+00, 0.00000000e+00,
       6.75579605e-14, 6.32909415e-14, 8.58661661e-01])

## 2. method using `cvxopt`
아래에서 사용된 `solvers.qp` 에 대한 설명은 [이곳](https://courses.csail.mit.edu/6.867/wiki/images/a/a7/Qp-cvxopt.pdf)을 참고하자:

In [7]:
from cvxopt import matrix, solvers

def MinVol_cvxopt(covmat, lb, ub):
    sz = len(covmat)

    P = matrix(np.array(covmat), tc='d')
    q = matrix(np.zeros(sz), tc='d')  # q는 0 벡터

    lb_diag = np.diag(np.repeat(-1, sz))
    ub_diag = np.diag(np.repeat(+1, sz))

    G = matrix(np.concatenate((lb_diag, ub_diag)), tc='d')
    h = matrix(np.concatenate((np.repeat(-lb, sz), np.repeat(ub, sz))), tc='d')

    A = matrix(np.repeat(1, sz), tc='d').T
    b = matrix(1.0, tc='d')

    solution = solvers.qp(P, q, G, h, A, b)
    return np.array(solution['x'])

In [8]:
MinVol_cvxopt(cov, 0, 1)

     pcost       dcost       gap    pres   dres
 0:  2.1092e-01 -8.2183e+00  3e+01  1e+00  7e-14
 1:  2.4141e-01 -2.9798e+00  4e+00  5e-02  4e-14
 2:  2.2681e-01 -1.9065e+00  3e+00  3e-02  3e-14
 3:  1.1860e+00  1.1427e+00  3e+00  2e-02  9e-14
 4:  3.0345e+00  3.0919e+00  3e-01  1e-03  2e-13
 5:  3.2517e+00  3.1884e+00  9e-02  9e-05  4e-13
 6:  3.2289e+00  3.2275e+00  2e-03  1e-06  2e-13
 7:  3.2281e+00  3.2281e+00  2e-05  1e-08  6e-14
 8:  3.2281e+00  3.2281e+00  2e-07  1e-10  3e-14
Optimal solution found.


array([[ 1.41338521e-01],
       [ 1.63191891e-09],
       [-4.01945284e-11],
       [ 3.10804555e-14],
       [ 2.34023054e-08],
       [ 1.60346753e-09],
       [ 8.58661453e-01]])